In [1]:
from bs4 import BeautifulSoup
import requests
import json
import os
import time

from csv import writer
from tqdm import tqdm, trange
from string import digits
from os.path import exists

## National Park

Source: https://www.nps.gov/index.htm

ATTENTION: DO NOT RUN IF YOU WANT TO USE THIS DATASET!

In [23]:
class NationalSite:
    def __init__(self, name, state, title, description):
        # Init instance of national site
        self.name = name
        self.state = state
        self.title = title
        self.description = description
    
    def info(self):
        # Return a string representation of itself
        rep = '"' + '","'.join([self.name, self.state, self.title, self.description]) + '"\n'    
        return rep
    
    def write_csv(self):
        try:
            with open("documents.csv", "x") as file:
                file.write("Name,State,Title,Description\n")
        except:
            pass
        with open("documents.csv", "a", encoding='utf-8') as file:
            file.write(self.info())

In [69]:
def build_state_url_dict():
    response = requests.get("https://www.nps.gov").text
    soup = BeautifulSoup(response, 'html.parser')

    selector = 'ul[role="menu"] > li > a'
    state_urls = soup.select(selector)
    state_dict = {
        state_url.string.lower(): 'https://www.nps.gov' + \
            state_url["href"] for state_url in state_urls
    }
    return state_dict

In [70]:
def get_sites_urls_for_state(state_url):
    response = requests.get(state_url).text
    soup = BeautifulSoup(response, 'html.parser')

    selector = "h3 > a"
    site_hrefs = soup.select(selector)
    site_urls = ["https://www.nps.gov" + site_href["href"] for site_href in site_hrefs]

    return site_urls

In [71]:
def get_site_instance(state, site_url):
    response = requests.get(site_url).text
    soup = BeautifulSoup(response, 'html.parser')

    name = soup.find_all("a", {"class": "Hero-title"})[0].get_text()
    title = soup.find_all("div", {"class": "Component text-content-size text-content-style"})[0].find_all("h1")[0].get_text()
    description = soup.find_all("div", {"class": "Component text-content-size text-content-style"})[0].find_all("p")[0].get_text()
    
    nationalsite = NationalSite(name.replace('"', ""), 
                                state.replace('"', ""), 
                                title.replace('"', ""), 
                                description.replace('"', ""))
    
    return nationalsite

In [72]:
# save to csv
state_dict = build_state_url_dict()
for state, state_url in tqdm(state_dict.items()):
    site_urls = get_sites_urls_for_state(state_url)
    for site_url in site_urls:
        site = get_site_instance(state, site_url)
        site.write_csv()

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [01:18<00:00,  1.39s/it]


## Tourist attractions in US cities

Source: https://www.city-data.com/articles/


In [2]:
class TouristSite:
    def __init__(self, name, state, title, description):
        # Init instance of national site
        self.name = name
        self.state = state
        self.title = title
        self.description = description
    
    def info(self):
        # Return a string representation of itself
        rep = '"' + '","'.join([self.name, self.state, self.title, self.description]) + '"\n'    
        return rep
            
    def write_single_csv(self):
        filename = './data/' + self.state + '_documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())
            
    def write_whole_csv(self):
        filename = 'documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())

In [3]:
def get_state_url_dict(source = "https://www.city-data.com/articles/"):
    # get a dict of state page name and its urls to state page
    response = requests.get(source).text
    soup = BeautifulSoup(response, 'html.parser')
    
    selector = 'div[id="tabs_by_category"] > ul[class="tab-list tab-list-short"] > li > a'
    state_urls = soup.select(selector)
    state_dict = {
        state_url['href'].split('/')[-1].split('.')[0]: \
            'https:' + state_url['href'] \
            for state_url in state_urls
    }
    return state_dict # key: state_page_name, value: url

In [4]:
def cache_sites_for_state_page(state_page_name, state_url):
    # cache the sites in a state page and store it into json file, like Florida6.json
    response = requests.get(state_url).text
    soup = BeautifulSoup(response, 'html.parser')
    
    # read the site pages
    selector = 'div[id="listing"] > div[class="col-md-4"] > a'
    site_urls = soup.select(selector)
    print(f'-> Reading site pages from state page <{state_page_name}>...')
    site_dict = {}
    for site_url in tqdm(site_urls):
        key = 'https://www.city-data.com' + site_url['href']
        value = requests.get('https://www.city-data.com' + site_url['href']).text
        site_dict[key] = value
        time.sleep(1) # add a gap between two request to prevent failure

    # write to cache file
    print(f'-> Caching state page <{state_page_name}>...')
    filename = './cache/' + state_page_name + '.json'
    with open(filename, 'w') as json_file:
        json.dump(site_dict, json_file, indent=4)
    print('Done.')
    
    return site_urls # a list of site urls in state page
    
def get_cache_sites_for_state_page(state_page_name):
    # get a list of site urls given the state page
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
    except:
        return None
    
    site_list = list(site_dict.keys())
    return site_list

In [5]:
def get_cache_site_page(state_page_name, site_url):
    # helper function: get response of a site page, return None if not exist
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
            response = site_dict[site_url]
    except:
        return None
    
    return response

def get_site_instance(state_page_name, site_url):
    # get the site instance for a site page
    response = get_cache_site_page(state_page_name, site_url)
    soup = BeautifulSoup(response, 'html.parser')

    state = state_page_name.translate(str.maketrans('', '', digits))
    title = soup.select('h1[class="city"] > span')[0].contents[0]
    paragraphs = [p.contents[0] for p in soup.select('div[class="well"] > p')]
    description = ' '.join(paragraphs)
    if title.count(' - ') == 2:
        name = title.split(' - ')[0]
    elif title.count(', ') == 2:
        name = title.split(', ')[0]
    else:
        name = title
    
    touristsite = TouristSite(name.replace('"', ""),
                              state.replace('"', ""), 
                              title.replace('"', ""), 
                              description.replace('"', ""))
    
    return touristsite

In [6]:
# --- Main --- #
state_dict = get_state_url_dict()

# cache all sites
# RUN THIS ONCE ONLY!!!
print("--- START CACHING FROM SOURCE WEBSITE ---")
for state_page_name, state_url in tqdm(state_dict.items()):
    filename = './cache/' + state_page_name + '.json'
    if not state_page_name[-1].isdigit() and not exists(filename):
        cache_sites_for_state_page(state_page_name, state_url)
print("---> CACHING DONE!")

  0%|                                                                                           | 0/77 [00:00<?, ?it/s]

--- START CACHING FROM SOURCE WEBSITE ---



  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

-> Reading site pages from state page <California3>...



 22%|█████████████████▊                                                               | 66/300 [01:22<04:54,  1.26s/it]


...

-> Reading site pages from state page <California4>...



 22%|█████████████████▊                                                               | 66/300 [01:22<04:54,  1.26s/it]


 44%|███████████████████████████████████▏                                            | 132/300 [02:44<03:29,  1.25s/it]


 12%|█████████▍                                                                       | 9/77 [10:27<1:18:58, 69.69s/it]


ConnectionError: HTTPSConnectionPool(host='www.city-data.com', port=443): Max retries exceeded with url: /articles/Hollywood-Hills.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000226A345F820>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [7]:
# write to the csv file
print("--- START WRITING FILES TO DOC DATA ---")
for state_page_name, state_url in tqdm(state_dict.items()):
    try:
        site_list = get_cache_sites_for_state_page(state_page_name)
        print(f"Writing sites from <{state_page_name}> to csv file...")
        filename = './data/' + state_page_name + '_documents.csv'
        if not exists(filename):
            for site_url in tqdm(site_list):
                site = get_site_instance(state_page_name, site_url)
                site.write_single_csv()
                site.write_whole_csv()
    except:
        pass

  0%|                                                                                           | 0/77 [00:00<?, ?it/s]

--- START WRITING FILES TO DOC DATA ---


  1%|█                                                                                  | 1/77 [00:00<00:33,  2.24it/s]

Writing sites from <Alaska> to csv file...


  3%|██▏                                                                                | 2/77 [00:00<00:29,  2.55it/s]

...

Writing sites from <California4> to csv file...
Writing sites from <California5> to csv file...
Writing sites from <California6> to csv file...
Writing sites from <California7> to csv file...
Writing sites from <California8> to csv file...
Writing sites from <California9> to csv file...
Writing sites from <California10> to csv file...
Writing sites from <California11> to csv file...
Writing sites from <Colorado> to csv file...
Writing sites from <Colorado2> to csv file...
Writing sites from <Connecticut> to csv file...
Writing sites from <District-of-Columbia> to csv file...
Writing sites from <Delaware> to csv file...
Writing sites from <Florida> to csv file...
Writing sites from <Florida2> to csv file...
Writing sites from <Florida3> to csv file...
Writing sites from <Florida4> to csv file...
Writing sites from <Florida5> to csv file...
Writing sites from <Florida6> to csv file...
Writing sites from <Georgia> to csv file...
Writing sites from <Georgia2> to csv file...
Writing sites f


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]
 97%|███████████████████████████████████████████████████████████████████████████████▊  | 75/77 [09:53<01:22, 41.21s/it]
0it [00:00, ?it/s]


Writing sites from <Minnesota> to csv file...
Writing sites from <Missouri> to csv file...
Writing sites from <Mississippi> to csv file...
Writing sites from <Montana> to csv file...
Writing sites from <North-Carolina> to csv file...
Writing sites from <North-Dakota> to csv file...
Writing sites from <Nebraska> to csv file...
Writing sites from <New-Hampshire> to csv file...
Writing sites from <New-Jersey> to csv file...
Writing sites from <New-Mexico> to csv file...
Writing sites from <Nevada> to csv file...
Writing sites from <Nevada2> to csv file...
Writing sites from <New-York> to csv file...
Writing sites from <New-York2> to csv file...
Writing sites from <New-York3> to csv file...
Writing sites from <Ohio> to csv file...
Writing sites from <Oklahoma> to csv file...
Writing sites from <Oregon> to csv file...
Writing sites from <Pennsylvania> to csv file...
Writing sites from <Pennsylvania2> to csv file...
Writing sites from <Rhode-Island> to csv file...
Writing sites from <South-C

100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [09:53<00:00,  7.71s/it]

Writing sites from <Wyoming> to csv file...


In [12]:
state_dict

{'Alaska': 'https://www.city-data.com/articles/Alaska.html',
 'Alabama': 'https://www.city-data.com/articles/Alabama.html',
 'Arkansas': 'https://www.city-data.com/articles/Arkansas.html',
 'Arizona': 'https://www.city-data.com/articles/Arizona.html',
 'Arizona2': 'https://www.city-data.com/articles/Arizona2.html',
 'Arizona3': 'https://www.city-data.com/articles/Arizona3.html',
 'California': 'https://www.city-data.com/articles/California.html',
 'California2': 'https://www.city-data.com/articles/California2.html',
 'California3': 'https://www.city-data.com/articles/California3.html',
 'California4': 'https://www.city-data.com/articles/California4.html',
 'California5': 'https://www.city-data.com/articles/California5.html',
 'California6': 'https://www.city-data.com/articles/California6.html',
 'California7': 'https://www.city-data.com/articles/California7.html',
 'California8': 'https://www.city-data.com/articles/California8.html',
 'California9': 'https://www.city-data.com/articles/